In [ ]:
# Import standard libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

# Import sklearn libraries for model evaluation and data splitting
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier

# Import PyTorch utilities for data handling
from torch.utils.data import DataLoader, Dataset

# Import transformers for model and tokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertModel, BertTokenizer, CamembertModel, CamembertTokenizer

# Import custom modules
import load_data
from utils_ipu import *
from training_audio_model_ipu import *

# Set device to GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

# Load data
transcr_path='paco-cheese/transcr'
data=load_data.load_all_ipus(folder_path=transcr_path,load_words=True)

In [ ]:
# Convert the 'data' variable into a pandas DataFrame
df = pd.DataFrame(data)

# Generate the target variable 'y'
y = create_y(df)

# Display
print(y.count(1))

In [ ]:
# Define the path to the audio files
audio_files_path = 'paco-cheese/audio/2_channels/'

# Extract audio segments from the audio files based on the information in 'data'
audio_segments = extract_audio_segments(data, audio_files_path)

# Display
audio_segments

In [ ]:
#Display
print(len(audio_segments))
print(len(y))

In [ ]:
# For each segment, extract features and store the extracted features in a numpy array 'X'
X = np.array([extract_features(segment) for segment in audio_segments])

In [ ]:
# Create an AudioDataset from the features 'X' and labels 'y'
dataset = AudioDataset(X, y)

# Set a seed for reproducibility
seed = 42

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Create AudioDataset objects for the training and test sets
train_dataset = AudioDataset(X_train, y_train)
test_dataset = AudioDataset(X_test, y_test)

# Create DataLoaders for these datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print the length of 'X' and 'y_test'
print(len(X))
print(len(y_test))

In [ ]:
# Set the device to GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Initialize variables for early stopping
patience=5
best_val=0
best_model=None

# Initialize the model and move it to the device
model = AudioCNN().to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Set the number of epochs for training
num_epochs = 10

# Define class weights for the loss function
class_weights = torch.tensor([1.0, 2], device=device)

# Define the loss function with class weights
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    
    # Loop over the training data
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move inputs and labels to the device

        optimizer.zero_grad()  # Zero the gradients

        outputs = model(inputs)  # Forward pass
        loss = loss_fn(outputs, labels)  # Compute the loss

        loss.backward()  # Backward pass
        optimizer.step()  # Update the weights

    # Evaluation
    model.eval()  # Set the model to evaluation mode
    total = 0
    correct = 0
    with torch.no_grad():  # Disable gradient calculations
        # Loop over the test data
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move inputs and labels to the device

            outputs = model(inputs)  # Forward pass
            _, predicted = torch.max(outputs.data, 1)  # Get the predicted classes
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  # Count the number of correct predictions
            
    # Compute the F1 score
    all_preds_audio,all_labels=predition_model_audio(model,test_loader,device,proba=False)
    f1 = f1_score(all_labels, all_preds_audio)
    
    # Check for improvement in F1 score
    if f1>best_val:
        best_val=f1
        best_model=copy.deepcopy(model)
        patience=5
    else:
        patience-=1
    if patience==0:
        break  # Early stopping

    # Print the loss and accuracy for this epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {100 * correct / total:.2f}%')

# Save the model
torch.save(model, 'modele/model_audio')

In [ ]:
# Print the count of '0's in the target variable 'y'
print(y.count(0))

# Print the count of '1's in the target variable 'y'
print(y.count(1))

# Print the proportion of '0's in the target variable 'y'
print(y.count(0)/len(y))

In [ ]:
# Use the best model to make predictions on the test data
all_preds_audio, all_labels = predition_model_audio(best_model, test_loader, device, proba=False)

# Compute the F1 score of the model's predictions
f1 = f1_score(all_labels, all_preds_audio)

# Compute the confusion matrix of the model's predictions
conf_matrix = confusion_matrix(all_labels, all_preds_audio)

# Print the F1 score and confusion matrix
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

# Compute the total number of instances for each class
total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])

# Compute the number of correctly detected instances for each class
detected_class_0 = conf_matrix[0, 0]  # True positives for class 0
detected_class_1 = conf_matrix[1, 1]  # True positives for class 1

# Print the number of correctly detected instances for each class
print(f'Nombre d\'éléments de classe 0 détectés : {detected_class_0} sur {total_class_0}')
print(f'Nombre d\'éléments de classe 1 détectés : {detected_class_1} sur {total_class_1}')